In [23]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [24]:
TRAIN = pd.read_csv(r'train.csv')
TEST = pd.read_csv(r'test.csv')

In [35]:
train=TRAIN.copy()
test=TEST.copy()
submit=test.filter(['UID'],axis=1)

In [36]:
train.head()

,UID,ph_no,cvv,credit_card_number,job,email,url,country,emoji,name,...,qhUzJ,FpCOT,zEnW3,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state
0,T77962103,7059868985,673.0,2.131870e+14,Copy,guerrerodavid@example.org,http://garcia.org/,Gabon,📑,Cathy Cherry,...,🍣🏿🟠U👨@9.028568970658524@️bY‍t,N6♀rs%-0.17374199145886737%g2Fu7,dwLNC+-16.438253859793118+D🟠Mj👩,♀Y‍tt^-5.683708646092431^7b🎓🍣T,Q👩👨H🏾?4.495020766798911?p🎨dBV,JC🍣8Q!1.3976340938324077!Wo‍6t,QtgcM|-2.1616012233562616|srwW❇,♀👩🦐pq%7.619063527183258%👨Oet♀,‍🏿🟠👩X+-7.00274740118153+‍b🎨VK,Despair
1,O92591533,+1-288-810-2425x013,983.0,4.820050e+15,Interior and spatial designer,michaela47@example.org,http://ortiz.com/,Bermuda,🥷🏿,Austin Graham,...,🍣🏿🟠U👨@-40.77392373961654@️bY‍t,N6♀rs%3.836527507653148%g2Fu7,dwLNC+-20.503862282922828+D🟠Mj👩,♀Y‍tt^4.345119531921794^7b🎓🍣T,Q👩👨H🏾?-0.19904265803833332?p🎨dBV,JC🍣8Q!3.414072326617339!Wo‍6t,QtgcM|8.673900995107994|srwW❇,♀👩🦐pq%-0.6359934104694338%👨Oet♀,‍🏿🟠👩X+-11.039355040136286+‍b🎨VK,Worthlessness
2,O31883571,791.502.4387x7276,855.0,4.031430e+15,"Scientist, water quality",zhoffman@example.org,http://kelly.com/,Sao Tome and Principe,👩‍💻,Frank Smith,...,🍣🏿🟠U👨@32.83118091536586@️bY‍t,N6♀rs%6.673810456792015%g2Fu7,dwLNC+31.545003675536307+D🟠Mj👩,♀Y‍tt^-1.1189348865153141^7b🎓🍣T,Q👩👨H🏾?-2.109315804181183?p🎨dBV,JC🍣8Q!6.773526691273227!Wo‍6t,QtgcM|1.7184076239046218|srwW❇,♀👩🦐pq%-35.95525145090523%👨Oet♀,‍🏿🟠👩X+8.6752786215897+‍b🎨VK,Stress
3,M18080565,2522308761,806.0,3.036420e+13,Toxicologist,hprice@example.net,http://mann.info/,Korea,NaN,Austin Young,...,🍣🏿🟠U👨@-11.829454766573882@️bY‍t,N6♀rs%-10.87012476653127%g2Fu7,dwLNC+26.131133464440587+D🟠Mj👩,♀Y‍tt^2.1159115210320385^7b🎓🍣T,Q👩👨H🏾?7.373067390546165?p🎨dBV,JC🍣8Q!-0.2388097363537749!Wo‍6t,QtgcM|-5.42787813792933|srwW❇,♀👩🦐pq%24.501780969213996%👨Oet♀,‍🏿🟠👩X+-0.8456272162320895+‍b🎨VK,Fatigue
4,Q74073854,683-521-2001x423,424.0,4.610210e+15,Software engineer,michael61@example.net,https://www.mitchell.biz/,Russian Federation,💆🏿‍♂️,Kimberly Wilson,...,🍣🏿🟠U👨@10.957430264267883@️bY‍t,N6♀rs%-1.5309050040841568%g2Fu7,dwLNC+-17.670060957130605+D🟠Mj👩,♀Y‍tt^0.132418797411758^7b🎓🍣T,Q👩👨H🏾?3.6960467337932292?p🎨dBV,JC🍣8Q!4.588960052905303!Wo‍6t,QtgcM|-3.735829327461521|srwW❇,♀👩🦐pq%18.194146730985878%👨Oet♀,‍🏿🟠👩X+10.404818391481196+‍b🎨VK,Tears


In [37]:
columns=list(train.columns)
print(len(columns))

111


In [28]:
print(sum([True for i,row in train.iterrows() if any(row.isnull())]))

140000


In [38]:
def slice_column(df,column,split):
    df[column]=df[column].astype(str)
    if split=='-':
        split_result = df[column].str.extract(r'(.*?)-(-?\d+\.\d+)-(.*)')
    else:
        split_result = df[column].str.split(split, expand=True)
    if split_result.shape[1] == 3:
        df[column]=split_result.iloc[:,1].astype(float)
    return df

splitlist="^|%#|%|#/%+?#-^|?|$|^!%?-^?|/$/+?||$|$$^+|?!!##-@!?@$|++!-$!+/^^|/|^/|+^/|!?@||?-#$|-!-^-$!@%+^?!|%+"
print(len(splitlist))


100


In [39]:
def clean_data(df):
    for i in range(100):
        df=slice_column(df,columns[i+10],splitlist[i])
    return df

train = clean_data(train)
test = clean_data(test)

In [40]:
def clean(df,columnlist):
  df.dropna(subset=columnlist,inplace=True)
  df.reset_index(drop=True,inplace=True)
  return df

def drop_column(df,columnlist):
  for column in columnlist:
    df.drop(column, axis=1, inplace=True)
  return df

def preprocess(df,clean_req=0):
  if clean_req:
    df=clean(df,['job','country','emoji','email','url'])
  df['email'] = df['email'].astype(str)
  df['url'] = df['url'].astype(str)
  df[['username', 'mail']] = df['email'].str.rsplit("@",1, expand=True)
  df[['protocol','url']]= df['url'].str.split(":",1,expand=True)
  df[['url','extension']]= df['url'].str.rsplit(".",1,expand=True)
  df=drop_column(df,['UID','cvv','ph_no','name','username','email','url'])
  return df

In [41]:
train=preprocess(train,1)
test=preprocess(test)
train.head()

C:\Users\Sreeya Srikanth\AppData\Local\Temp\ipykernel_11680\2915680199.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['username', 'mail']] = df['email'].str.rsplit("@",1, expand=True)
C:\Users\Sreeya Srikanth\AppData\Local\Temp\ipykernel_11680\2915680199.py:17: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['protocol','url']]= df['url'].str.split(":",1,expand=True)
C:\Users\Sreeya Srikanth\AppData\Local\Temp\ipykernel_11680\2915680199.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['url','extension']]= df['url'].str.rsplit(".",1,expand=True)
C:\Users\Sreeya Srikanth\AppData\Local\Temp\ipykernel_11680\2915680199.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.rspl

,credit_card_number,job,country,emoji,K3Ll9,19rjS,yeIIP,Bw1V5,5k16L,e2l5S,...,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state,mail,protocol,extension
0,2.131870e+14,Copy,Gabon,📑,-2.150757,-47.000246,7.191216,12.185065,-1.052368,12.327996,...,-5.683709,4.495021,1.397634,-2.161601,7.619064,-7.002747,Despair,example.org,http,org/
1,4.820050e+15,Interior and spatial designer,Bermuda,🥷🏿,1.190290,-6.230500,-17.179968,23.406359,3.302922,15.152568,...,4.345120,-0.199043,3.414072,8.673901,-0.635993,-11.039355,Worthlessness,example.org,http,com/
2,4.031430e+15,"Scientist, water quality",Sao Tome and Principe,👩‍💻,4.560737,21.534617,45.301605,-18.885383,-4.262870,15.240195,...,-1.118935,-2.109316,6.773527,1.718408,-35.955251,8.675279,Stress,example.org,http,com/
3,4.610210e+15,Software engineer,Russian Federation,💆🏿‍♂️,-6.973737,-12.733164,3.526667,6.197023,-4.558279,1.983331,...,0.132419,3.696047,4.588960,-3.735829,18.194147,10.404818,Tears,example.net,https,biz/
4,2.131350e+14,Health promotion specialist,Congo,🧍🏾‍♀️,1.490146,10.792539,-11.607395,NaN,-7.346593,21.987029,...,1.845671,-2.799491,-1.243306,-2.265135,-26.643237,-1.105436,Nausea,example.org,https,net/


In [42]:
le = preprocessing.LabelEncoder()

def recode_data(df):
  for c in ['job','country','emoji','mail','protocol','extension']:
    df[c]=le.fit_transform(df[c])
  return df

In [43]:
train=recode_data(train)
test=recode_data(test)
train['state']=le.fit_transform(train['state'])

In [44]:
def impute_values(df):
    for c in df.columns:
        if df[c].isnull().any():
            df[c].fillna(df[c].mean(), inplace=True)
    return df

In [45]:
train=impute_values(train)
test=impute_values(test)

In [46]:
train.describe()

,credit_card_number,job,country,emoji,K3Ll9,19rjS,yeIIP,Bw1V5,5k16L,e2l5S,...,ASDn5,vF2is,pZijn,WUc3c,sCIyG,qaERi,state,mail,protocol,extension
count,1.336080e+05,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,...,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000,133608.000000
mean,3.726935e+17,319.209808,120.503802,1754.150253,0.406558,0.695369,-0.234812,0.051346,0.077720,1.068052,...,-0.187066,-0.201080,0.303929,0.021093,0.988631,-0.007979,4.509326,1.002088,0.499049,1.496250
std,1.232395e+18,184.369389,69.936984,1011.883376,4.187378,17.239268,18.461962,16.719290,4.178623,19.157364,...,4.214237,4.134768,4.098821,4.208693,19.585398,4.124185,2.871771,0.815171,0.500001,1.117811
min,6.040020e+10,0.000000,0.000000,0.000000,-17.378663,-77.636372,-88.816744,-84.533117,-20.864162,-88.800077,...,-18.823306,-18.528236,-18.311135,-19.833364,-97.657266,-18.691201,0.000000,0.000000,0.000000,0.000000
25%,1.800030e+14,160.000000,60.000000,877.000000,-2.389618,-10.640539,-12.410518,-11.133906,-2.709930,-11.723922,...,-3.004360,-2.984184,-2.441459,-2.792158,-12.076414,-2.767997,2.000000,0.000000,0.000000,1.000000
50%,3.515015e+15,319.000000,120.000000,1756.000000,0.406558,0.695369,-0.234812,0.051346,0.077720,1.068052,...,-0.187066,-0.201080,0.303929,0.021093,0.988631,-0.007979,5.000000,1.000000,0.000000,1.000000
75%,4.670368e+15,479.000000,181.000000,2630.000000,3.195914,12.211279,12.021515,11.027391,2.866756,13.859381,...,2.641277,2.555254,3.058478,2.827434,13.845614,2.733762,7.000000,2.000000,1.000000,2.000000
max,4.999930e+18,638.000000,242.000000,3508.000000,19.441730,81.743863,77.957198,74.036401,18.870930,86.574856,...,19.636270,18.474009,19.516091,18.421846,102.783205,18.189089,9.000000,2.000000,1.000000,4.000000


In [47]:
y=train.filter(['state'],axis=1)
x=drop_column(train,['state'])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)

In [48]:
sc = preprocessing.StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

test=sc.fit_transform(test)

In [49]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(xtrain, ytrain)

c:\Users\Sreeya Srikanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=0)

In [50]:
ypred = classifier.predict(xtest)
print(ypred)

[3 8 1 ... 9 7 6]


In [51]:
accuracy=accuracy_score(ytest, ypred)
print(accuracy)

0.4319661701968416


In [52]:
yfinal= classifier.predict(test)
yfinal=le.inverse_transform(yfinal)
print(yfinal)
submit['state']=yfinal
submit.to_csv('submission.csv',index=False)

['Fatigue' 'Stress' 'Fatigue' ... 'Tears' 'Despair' 'Tears']
